In [1]:
# step4_hybrid_fusion.py
"""
FUSION HYBRIDE : Triplets fiables + Paires résiduelles non chevauchantes

Objectif :
- Garder TOUS les triplets de haute qualité (score ≥ 0.60)
- Ajouter uniquement les paires (GitHub↔LinkedIn, etc.) :
    • avec score ≥ 0.65
    • et dont AUCUN des profils n’est déjà dans un triplet

Résultat : une base unifiée plus précise que les paires seules, plus complète que les triplets seuls.
"""

import json
from pathlib import Path

def main():
    output_dir = Path("output")
    
    # -----------------------------
    # 1. Charger les triplets (haute précision)
    # -----------------------------
    try:
        with open(output_dir / "unified_triplets.json", "r", encoding="utf-8") as f:
            triplets = json.load(f)
    except FileNotFoundError:
        print("⚠️ unified_triplets.json non trouvé. Crée-le avec step3_triplet_matching.py")
        triplets = []

    # Extraire les IDs de profils déjà utilisés dans des triplets
    profiles_in_triplets = set()
    for triplet in triplets:
        for p in triplet["profiles"]:
            # On suppose que chaque profil a un identifiant unique ou on utilise son contenu
            # Ici, on utilise une combinaison plateforme + username/email comme clé robuste
            key = (
                p["platform"],
                p.get("username", "").lower(),
                p.get("email", "").lower() if p.get("email") else ""
            )
            profiles_in_triplets.add(key)

    print(f"✅ {len(triplets)} triplets chargés → {len(profiles_in_triplets)} profils protégés")

    # -----------------------------
    # 2. Charger les paires/clusters (de la méthode par paires)
    # -----------------------------
    try:
        with open(output_dir / "unified_profiles.json", "r", encoding="utf-8") as f:
            unified_pairs = json.load(f)
    except FileNotFoundError:
        print("⚠️ unified_profiles.json non trouvé. Crée-le avec step3_weighted_matching.py")
        unified_pairs = []

    # -----------------------------
    # 3. Filtrer les paires résiduelles
    # -----------------------------
    residual_pairs = []
    added_profiles = set()  # pour éviter les doublons dans les paires elles-mêmes

    for cluster in unified_pairs:
        profiles = cluster["profiles"]
        platforms = {p["platform"] for p in profiles}

        # On ne garde que les clusters de taille 2 (paires)
        if len(profiles) != 2:
            continue

        # Vérifier si cette paire est déjà couverte par un triplet
        overlap = False
        for p in profiles:
            key = (
                p["platform"],
                p.get("username", "").lower(),
                p.get("email", "").lower() if p.get("email") else ""
            )
            if key in profiles_in_triplets:
                overlap = True
                break

        if overlap:
            continue  # ignorer : déjà dans un triplet

        # Extraire le score (si présent) ou estimer via similarité
        score = cluster.get("score")
        if score is None:
            # Si le fichier unified_profiles.json ne contient pas de score,
            # on suppose qu’il vient de la méthode par paires → utiliser un seuil basé sur la logique métier
            # Ici, on accepte toutes les paires non chevauchantes avec seuil = 0.65
            score = 0.70  # valeur par défaut haute (car méthode par paires déjà filtrée)

        # Appliquer seuil strict pour les paires résiduelles
        if score >= 0.65:
            # Vérifier aussi que les deux profils ne sont pas déjà utilisés dans une autre paire ajoutée
            keys = []
            for p in profiles:
                key = (
                    p["platform"],
                    p.get("username", "").lower(),
                    p.get("email", "").lower() if p.get("email") else ""
                )
                keys.append(key)

            if any(k in added_profiles for k in keys):
                continue  # conflit → ignorer

            added_profiles.update(keys)
            residual_pairs.append({
                "unified_id": f"pair_{len(residual_pairs):05d}",
                "score": float(score),
                "profiles": profiles
            })

    print(f"✅ {len(residual_pairs)} paires résiduelles ajoutées (score ≥ 0.65, sans chevauchement)")

    # -----------------------------
    # 4. Fusionner et sauvegarder
    # -----------------------------
    hybrid_unified = []

    # Ajouter d'abord les triplets (haute priorité)
    for t in triplets:
        hybrid_unified.append(t)

    # Puis les paires résiduelles
    for p in residual_pairs:
        hybrid_unified.append(p)

    # Sauvegarde
    output_path = output_dir / "unified_hybrid.json"
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(hybrid_unified, f, indent=2, ensure_ascii=False)

    # Stats finales
    total_identities = len(hybrid_unified)
    total_profiles = sum(len(u["profiles"]) for u in hybrid_unified)
    triplets_count = sum(1 for u in hybrid_unified if len(u["profiles"]) == 3)
    pairs_count = sum(1 for u in hybrid_unified if len(u["profiles"]) == 2)

    print(f"\n🎉 Fusion hybride terminée :")
    print(f"   → {triplets_count} triplets")
    print(f"   → {pairs_count} paires résiduelles")
    print(f"   → {total_identities} identités unifiées")
    print(f"   → {total_profiles} profils couverts")
    print(f"   → Résultat sauvegardé dans '{output_path}'")

    # Optionnel : afficher 2 exemples
    if hybrid_unified:
        print(f"\n🔍 Exemple de triplet :")
        ex = next((u for u in hybrid_unified if len(u["profiles"]) == 3), None)
        if ex:
            p = ex["profiles"][0]
            print(f"   {p.get('fullName', '—')} ({p['platform']}) + 2 autres")

        print(f"🔍 Exemple de paire résiduelle :")
        ex = next((u for u in hybrid_unified if len(u["profiles"]) == 2), None)
        if ex:
            p1, p2 = ex["profiles"]
            print(f"   {p1.get('fullName', '—')} ({p1['platform']}) ↔ {p2.get('fullName', '—')} ({p2['platform']})")

if __name__ == "__main__":
    main()

✅ 388 triplets chargés → 1155 profils protégés
✅ 285 paires résiduelles ajoutées (score ≥ 0.65, sans chevauchement)

🎉 Fusion hybride terminée :
   → 388 triplets
   → 285 paires résiduelles
   → 673 identités unifiées
   → 1734 profils couverts
   → Résultat sauvegardé dans 'output/unified_hybrid.json'

🔍 Exemple de triplet :
   Salma Bouziane (github) + 2 autres
🔍 Exemple de paire résiduelle :
   SAID ABDEREMANE (github) ↔ Said ABDEREMANE (linkedin)
